# Imports

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt

import os
import subprocess

# Utils

In [ ]:
N = np.arange(100, 2050, step=np.int32(50))

graph_types = ['complete', 'path_end', 'path_middle', 'tree', 'lollipop']
files = pd.DataFrame({
    'matrix': ['matrices/' + type + '.csv' for type in graph_types],
    'result': ['results/' + type + '.csv' for type in graph_types],
    'savefig': ['plot_images/' + type + '.png' for type in graph_types]
}, index=graph_types)



def _adj_matrix(G: nx.Graph(), export: str = None, v0: callable or int = 0):
    adj_m = nx.to_pandas_adjacency(G, dtype=np.int32)

    if export:
        with open(export, 'a') as file:
            n = len(G.nodes)
            if callable(v0): file.write(f"{n} {v0(n)}\n")
            else: file.write(f"{n} {int(v0)}\n")
        adj_m.to_csv(export, mode='a', sep=' ', index=False, header=False)

    return adj_m



def _plot_result(df: pd.DataFrame, graph: str, F: list, F_names: list, save: str = None):
    assert all([callable(f) for f in F]), "All elements in functions must be callable!"
    assert len(F_names) == len(F), "F_names must have the same length as F!"

    fig = plt.figure(figsize=(10, 10))
    nf = len(F)
    axs = [plt.subplot2grid((2, nf), (0, 0), colspan=nf)]
    plt.subplots_adjust(hspace=0.2, wspace=0.15)

    axs[0].set_title(f"{graph} graph random walk cover times")
    for (n, times) in df.iteritems():
        axs[0].scatter([n] * times.shape[0], times, color='blue', s=5)
    axs[0].scatter(df.columns, df.mean(), color='red', s=20)
    axs[0].set_xticklabels(df.columns, rotation=45)
    axs[0].set_xlabel("$n$ - graph size")
    axs[0].set_ylabel("$T(n)$ - cover time")

    mean_vals = df.mean()
    for i in range(nf):
        ax_i = i + 1
        axs.append(plt.subplot2grid((2, nf), (1, i), colspan=1))
        axs[ax_i].set_xlabel("n - graph size")
        axs[ax_i].set_ylabel("$\\frac{E(T(n))}{f(n)}$" + f"\t$f = {F_names[i]}$")

        plot_vals = mean_vals / F[i](mean_vals.index.values.astype(np.int64))
        print(f"Plot values < 0: {(plot_vals <= 0).any()}")
        axs[ax_i].grid()
        axs[ax_i].scatter(mean_vals.index, plot_vals, color="blue", s=10)
        axs[ax_i].set_xticklabels(mean_vals.index, rotation=45)
        plot_min = min(plot_vals)
        plot_max = max(plot_vals)
        axs[ax_i].set_ylim(plot_min + (plot_min - plot_max), plot_max + (plot_max - plot_min))

    if save: plt.savefig(save)

    plt.show();
    


def _nln(n):
    return n * np.log(n)

def _nlnln(n):
    return n * np.log(n) ** 2

def _square(n):
    return n ** 2

def _cube(n):
    return n ** 3

# Experiments

In [ ]:
subprocess.run("g++ -o hw4 hw4_src.cpp".split())

* Complete graph

In [ ]:
if os.path.exists(files.matrix['complete']): os.remove(files.matrix['complete'])
for n in N:
    complete = nx.complete_graph(n, create_using=nx.Graph())
    _adj_matrix(complete, export=files.matrix['complete'])

subprocess.run(['./hw4.exe', 'complete'])

In [ ]:
complete_results = pd.read_csv(files.result['complete'], index_col=0)
_plot_result(complete_results, graph='Complete', F=[_nln], F_names=['n*ln(n)'], save=files.savefig['complete'])
complete_results

* Path graph (starting at an end)

In [ ]:
if os.path.exists(files.matrix['path_end']): os.remove(files.matrix['path_end'])
for n in N:
    path_end = nx.path_graph(n, create_using=nx.Graph())
    _adj_matrix(path_end, export=files.matrix['path_end'])

subprocess.run(['./hw4.exe', 'path_end'])

In [ ]:
path_end_results = pd.read_csv(files.result['path_end'], index_col=0)
_plot_result(path_end_results, graph='Path (starting at an end)', F=[_square], F_names=['n^2'], save=files.savefig['path_end'])
path_end_results

* Path graph (starting in the middle)

In [ ]:
def _half(n):
    return n // 2

if os.path.exists(files.matrix['path_middle']): os.remove(files.matrix['path_middle'])
for n in N:
    path_middle = nx.path_graph(n, create_using=nx.Graph())
    _adj_matrix(path_middle, export=files.matrix['path_middle'], v0=_half)

subprocess.run(['./hw4.exe', 'path_middle'])

In [ ]:
path_middle_results = pd.read_csv(files.result['path_middle'], index_col=0)
_plot_result(path_middle_results, graph='Path (starting in the middle)', F=[_square], F_names=['n^2'], save=files.savefig['path_middle'])
path_middle_results

* Balanced tree graph

In [ ]:
if os.path.exists(files.matrix['tree']): os.remove(files.matrix['tree'])
for n in N:
    logn = int(np.ceil(np.log2(n)))
    tree = nx.balanced_tree(2, logn, create_using=nx.Graph())
    tree.remove_nodes_from(np.arange(n, len(tree.nodes)))
    _adj_matrix(tree, export=files.matrix['tree'])

subprocess.run(['./hw4.exe', 'tree'])

In [ ]:
tree_results = pd.read_csv(files.result['tree'], index_col=0)
_plot_result(tree_results, graph='Tree', F=[_nlnln], F_names=['n*ln(n)*ln(n)'], save=files.savefig['tree'])
tree_results

* Lollipop graph

In [ ]:
if os.path.exists(files.matrix['lollipop']): os.remove(files.matrix['lollipop'])
for n in N:
    clique_size = (2 * n) // 3
    path_size = n - clique_size
    lollipop = nx.lollipop_graph(clique_size, path_size, create_using=nx.Graph())
    _adj_matrix(lollipop, export=files.matrix['lollipop'])

subprocess.run(['./hw4.exe', 'lollipop'])

In [ ]:
lollipop_results = pd.read_csv(files.result['lollipop'], index_col=0)
_plot_result(lollipop_results, graph='Lollipop', F=[_cube], F_names=['n^3'], save=files.savefig['lollipop'])
lollipop_results